import csv
from collections import defaultdict

toJson = open('anime.json', 'w')
count = 0
toJson.write('[')
with open('anime.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        if row[0] == 'anime_id':
            continue
        
        
        genre_list = row[2].split(',')
        genres = '['
        for g in range(len(genre_list)):
            genre = genre_list[g].replace(" ", "")
            genres += '"'
            genres += genre
            if g != len(genre_list)-1:
                genres += '",'
            else:
                genres += '"]'
                
        rating = row[5]
        if rating == '':
            rating = -1
        
        if (count != 0):
            toJson.write(',\n')
        
        toJson.write('{ "anime_id" : "' + str(row[0]) + '", "name" : "' + str(row[1]) + '", "genre" : '
              + str(genres) + ', "type" : "' + str(row[3]) + '", "episodes" : "'
              + str(row[4]) + '", "rating" : ' + str(rating) + ', "members" : ' + str(row[6]) + '}')
        
        count += 1
    
toJson.write(']')
toJson.close()

In [111]:
import json

rating = json.load(open('rating_200k.json'))
animedata = json.load(open('anime.json'))


In [2]:
from collections import defaultdict

watched = defaultdict(set)
rating_by_user = defaultdict(list)    # list of animes with rating that each user watched
rating_by_anime = defaultdict(list)    # list of dictionary with anime as keys, and containing user and ratings


for entry in rating:
    watched[entry['user_id']].add(entry['anime_id'])
    
    anime_dict = {}
    if (entry['rating'] != '-1'):
        anime_dict['anime_id'] = entry['anime_id']
        anime_dict['rating'] = entry['rating']
        rating_by_user[entry['user_id']].append(anime_dict)
    
    user_dict = {}
    if (entry['rating'] != '-1'):
        user_dict['rating'] = entry['rating']
        user_dict['user_id'] = entry['user_id'] 
        rating_by_anime[entry['anime_id']].append(user_dict)
    
    
    
    
    
   

In [3]:
genres = set()    # set of all genres
genreCount = defaultdict(int)     # count of each genre

for entry in animedata:
    for genre in entry['genre']:
        genres.add(genre)
        genreCount[genre] += 1

In [4]:
anime_view_dict = defaultdict(list)     # list of users that watched certain anime

for entry in rating:
    anime_id = entry['anime_id']
    user_id = entry['user_id']
    anime_view_dict[anime_id].append(user_id)

In [5]:
genre_by_anime = defaultdict(list)      # list of animes corresponding to certain genre

for entry in animedata: 
    for genre in entry['genre']:
        genre_by_anime[genre].append(entry['anime_id'])

In [6]:
average_rating_by_user = defaultdict(float)

for user in rating_by_user:
    
    animes = rating_by_user[user]
    total = 0
    count = 0
    
    for anime in animes:    
        total += float(anime['rating'])
        count += 1
            
    if count != 0:
        average_rating_by_user[user] = total / count
        
        

In [7]:
beta_u = defaultdict(float)

total = 0
count = 0
for user in average_rating_by_user:
    total += average_rating_by_user[user]
    count += 1
    
total_average_rating_byuser = total / count

for user in average_rating_by_user:
    beta_u[user] = average_rating_by_user[user] - total_average_rating_byuser

In [8]:
anime_average_rating = defaultdict(float)      # average rating of each anime

for entry in animedata:
    if (entry['rating'] != -1 and entry['rating'] != 0):
        anime_average_rating[entry['anime_id']] = entry['rating']

In [9]:
beta_i = defaultdict(float)

total = 0
count = 0
for anime in anime_average_rating:
    total += anime_average_rating[anime]
    count += 1
    
total_average_rating_byanime = total / count

for anime in anime_average_rating:
    beta_i[anime] = anime_average_rating[anime] - total_average_rating_byanime

In [10]:
alpha = anime_average_rating

In [11]:
# Make a train / validation set

rating_filtered = []
for entry in rating:
    if (entry['rating'] != '-1'):
        rating_filtered.append(entry)

In [12]:
# Approach 1

# rating_filtered

In [13]:
# Approach 2

import random

def scrambled(orig):
    dest = orig[:]
    random.shuffle(dest)
    return dest

rating_scrambled = scrambled(rating_filtered)

In [14]:
# Approach 3

rating_toModify = rating_filtered[:]

current_user_id = 1
rating_3 = []

checknumber = 150000

while(len(rating_toModify) > 3000):
    
    for j in range(0,len(rating_toModify)-1):

        if j >= len(rating_toModify)-1:
            break
        
        this_rating = rating_toModify[j]
        current_user_id_str = str(current_user_id)
        
        if this_rating['user_id'] == current_user_id_str:

            rating_3.append(this_rating)
            rating_toModify.pop(j)
            current_user_id += 1
            
        if int(this_rating['user_id']) > current_user_id:
            current_user_id += 1
            
        

    if current_user_id >= 2012:
        current_user_id = 1
    
remaining = rating_toModify[:]
rating_3 = rating_3 + remaining

In [15]:
# writeFile = open('ratingset_1.json', 'w')
# for entry in rating_filtered:
#     writeFile.write(str(entry))
    
# writeFile.close()

# writeFile = open('ratingset_2.json', 'w')
# for entry in rating_scrambled:
#     writeFile.write(str(entry))
    
# writeFile.close()

# writeFile = open('ratingset_3.json', 'w')
# for entry in rating_3:
#     writeFile.write(str(entry))
    
# writeFile.close()


In [16]:
rating_3 = rating_3[:150000]

In [17]:
len(rating_3)

150000

In [18]:
# Collaborative Filtering + Regression for rating prediction
# Collaborative Filtering for watch prediction (in case we can't figure out how to do rating prediction)
# Basic model for latent factor model
# How to optimize

### Graph
# rating according to genre
# rating according to types (Movie/OVA/TV Series)
# rating according to number of members
# Most popular word occuring in title
# Average number of members by genres


In [19]:
japanese_stopwords = {"a", "i", "u", "e", "o", "ka", "ki", "ku", "ke", "ko", "ga", "gi", "gu", "ge", "go",
                      "sa", "si", "su", "se", "so", "za", "zi", "zu", "ze", "zo",
                        "ta", "ti","tsu", "te", "to", "da", "di", "du", "de", "do", "na", "no", "ni","nu", "ne",
                      "ha", "hi", "hu", "he", "ho", "ba", "bi", "bu", "be", "bo", "ma", "mi", "mu", "me", "mo",
                      "ya", "yo","yu", "ra", "ri", "ru", "re", "ro", "wa", "wo", "n"}

number = {"1","2","3","4","5","6","7","8","9"}


In [20]:
# Most popular word occuring in title

import string
from nltk.stem.porter import *
from nltk.corpus import stopwords

ps = PorterStemmer()

popWordDict = defaultdict(int)


for entry in animedata:
    title = ''.join([c for c in entry['name'].lower() if not c in string.punctuation])
    for w in title.split():
        if w not in stopwords.words('english') and w not in japanese_stopwords and w not in number:
            popWordDict[w] += 1

popWord = []
for w in popWordDict:
    popWord.append([popWordDict[w],w])
popWord.sort()
popWord.reverse()
    

In [22]:
# Average number of members by genres
# Average ratings by genre

average_member_by_genres = defaultdict(int)
average_rating_by_genres = defaultdict(int)

for entry in animedata:
    members = entry['members']
    ratings = entry['rating']
    genres = []
    for genre in entry['genre']:
        average_member_by_genres[genre] += members
        average_rating_by_genres[genre] += ratings
        
for genre in average_member_by_genres:
    average_member_by_genres[genre] /=  genreCount[genre]
    average_rating_by_genres[genre] /= genreCount[genre]


In [23]:
typeCount = defaultdict(int)

for entry in animedata:
    typeCount[entry['type']] += 1

In [24]:
arbg = open('average_rating_by_genres.csv', 'w')

arbg.write("Genre\tAverage Rating\n")
for l in average_rating_by_genres:
    if l != '':
        arbg.write(l + "\t"+ str(average_rating_by_genres[l]) + "\n")
        
arbg.close()

In [25]:
pw = open('popular_words.csv', 'w')

pw.write("word\tcount\n")
for l in popWord:
    pw.write(str(l[1]) + "\t"+ str(l[0]) + "\n")
        
pw.close()

In [26]:
ambg = open('average_member_by_genres.csv', 'w')

ambg.write("Genre\tAverage Rating\n")
for l in average_member_by_genres:
    if l != '':
        ambg.write(l + "\t"+ str(average_member_by_genres[l]) + "\n")
        
ambg.close()

In [27]:
# Collaborative Filtering + Regression for rating prediction
# Collaborative Filtering for watch prediction (in case we can't figure out how to do rating prediction)
# Basic model for latent factor model
# How to optimize

In [262]:
training_set = rating_3[:75000]
validation_set = rating_3[75000:]

In [144]:
# somedict_anime = defaultdict(set)
# somedict_user = defaultdict(set)

# for l in training_set:
#     anime, user = l['anime_id'], l['user_id']
#     somedict_anime[user].add(anime)
#     somedict_user[anime].add(user)
    

In [214]:
# Rating prediction based on collaborative filtering + regression

def get_anime_users():
    #first get a dictionary of key: business user: set
    anime_user_dict = {}
    for review in training_set:
        anime = review['anime_id']
        user = review['user_id']
        if anime in anime_user_dict:
            user_set = anime_user_dict[anime]
            user_set.add(user)
        else:
            user_set = set()
            user_set.add(user)
            anime_user_dict[anime] = user_set
    return anime_user_dict

def get_user_anime():
    #first get a dictionary of key: business user: set
    user_anime_dict = {}
    for review in training_set:
        anime = review['anime_id']
        user = review['user_id']
        if user in user_anime_dict:
            anime_set = user_anime_dict[user]
            anime_set.add(anime)
        else:
            anime_set = set()
            anime_set.add(anime)
            user_anime_dict[user] = anime_set
    return user_anime_dict

anime_users_dict = get_anime_users()
user_anime_dict = get_user_anime()

def jaccard_sim_user(check_user, check_anime):
    #given a business, find the most similar business
    max_sim = 0.0
    count = 0

    if check_anime not in anime_users_dict: return 0
    else:
        users = anime_users_dict[check_anime]
        for user in users:
            if user == check_user: continue
            if check_user not in user_anime_dict: continue
            current_user_anime = user_anime_dict[user]
            check_user_anime = user_anime_dict[check_user]
            intersect = current_user_anime.intersection(check_user_anime)
            union = current_user_anime.union(check_user_anime)
            # print(intersect)
            # print(union)
            similarity = float(len(intersect))/len(union)
            if similarity > max_sim:
                max_sim = similarity
            #max_sim += similarity
            count += 1
            
                    
    return max_sim



In [236]:
import numpy

def feature(u,a):
    feat = [1]
    #feat.append(1 if jaccard_sim_user(u,a) > 0.01 else 0)
    feat.append(jaccard_sim_user(u,a))
#     if (similarity(u,a)):
#         feat.append(1)
#     else:
#         feat.append(0)
    return feat



X_train = [feature(d['user_id'],d['anime_id']) for d in training_set[:5000]+validation_set[:500]]
y_train = [float(d['rating']) for d in training_set[:5000]+validation_set[:500]]
X_validation = [feature(d['user_id'],d['anime_id']) for d in validation_set[:500]]
y_validation = [float(d['rating']) for d in validation_set[:500]]
theta, residuals, rank, s = numpy.linalg.lstsq(X_train,y_train)

In [237]:
theta

array([ 7.97982597,  1.42811111])

In [263]:
# X_train[:20]

In [238]:
def mse(X, y, theta, size):
    MSSum = 0.0
    for i in range(0,size):
        MSSum += numpy.square(int(y[i]) - numpy.dot(X[i], theta))
    return MSSum / size

print("MSE is " + str(mse(X_validation,y_validation, theta, len(X_validation))))
print("variance is " + str(numpy.var([int(d) for d in y_validation])))

MSE is 2.20230911455
variance is 1.966924


In [247]:
# Watch prediction based on collaborative filtering

anime_id_set = set()
user_id_set = set()
for l in rating:
    anime_id_set.add(l['anime_id'])
    user_id_set.add(l['user_id'])

In [269]:
import random

random_pair = []
count = 0

while(1):
    userid = random.sample(user_id_set,1)[0]
    animeid = random.sample(anime_id_set,1)[0]
    temp_dict = {}
    temp_dict['user_id'] = userid
    temp_dict['anime_id'] = animeid
    
    if 
    random_pair.append(temp_dict)
    
    count += 1
    
    if (count >= 75000):
        break

In [271]:
complete_validation_set = validation_set + random_pair

In [272]:
complete_validation_set[74990:75010]

[{'anime_id': '25157', 'rating': '9', 'user_id': '1689'},
 {'anime_id': '28713', 'rating': '5', 'user_id': '1720'},
 {'anime_id': '6261', 'rating': '5', 'user_id': '1822'},
 {'anime_id': '17655', 'rating': '8', 'user_id': '1835'},
 {'anime_id': '23273', 'rating': '9', 'user_id': '1853'},
 {'anime_id': '32370', 'rating': '7', 'user_id': '1938'},
 {'anime_id': '25805', 'rating': '7', 'user_id': '1984'},
 {'anime_id': '22319', 'rating': '2', 'user_id': '5'},
 {'anime_id': '31376', 'rating': '5', 'user_id': '123'},
 {'anime_id': '11553', 'rating': '8', 'user_id': '160'},
 {'anime_id': '22', 'user_id': '318'},
 {'anime_id': '1777', 'user_id': '1064'},
 {'anime_id': '21511', 'user_id': '1266'},
 {'anime_id': '22817', 'user_id': '569'},
 {'anime_id': '29916', 'user_id': '1554'},
 {'anime_id': '8086', 'user_id': '1455'},
 {'anime_id': '3335', 'user_id': '784'},
 {'anime_id': '12149', 'user_id': '1626'},
 {'anime_id': '11981', 'user_id': '934'},
 {'anime_id': '3100', 'user_id': '704'}]

In [273]:
def watch_prediction():
    
    count = 0
    total = 0
    
    for l in complete_validation_set:

        u,a = l['user_id'], l ['anime_id']
        
        if jaccard_sim_user(u,a):
            count += 1
            total += 1
            
        else:
            total += 1
            
    print("Accuracy of this model is")
    print(count * 1.0 / total)

watch_prediction()

Accuracy of this model is
0.83518


In [274]:
def watch_prediction2():
    
    true_pos = 0
    true_neg = 0
    
    for l in validation_set:
        u,a = l['user_id'], l['anime_id']
        if jaccard_sim_user(u,a):
            true_pos += 1
            
    for l in random_pair:
        u,a = l['user_id'], l['anime_id']
        if not jaccard_sim_user(u,a):
            true_neg += 1
            
    accuracy = (true_pos + true_neg) / len(complete_validation_set)
    print("Accuracy is ", accuracy)
    
watch_prediction2()

Accuracy is  0.6469133333333333
